In [2]:
import os
root = '/store2/dataset/coco'

subfolders = ['annotations']

anno_root = os.path.join(root, 'annotations')

anno_path = os.path.join(anno_root, 'instances_train2014.json')

import json
train = json.load(open(anno_path, 'r'))

In [1]:
import torch
torch.tensor([1,2]).cuda()

tensor([1, 2], device='cuda:0')

In [3]:
index = 100

for i in range(index):
    #print(train['images'][1])
    print(train['annotations'][i])

{'segmentation': [[312.29, 562.89, 402.25, 511.49, 400.96, 425.38, 398.39, 372.69, 388.11, 332.85, 318.71, 325.14, 295.58, 305.86, 269.88, 314.86, 258.31, 337.99, 217.19, 321.29, 182.49, 343.13, 141.37, 348.27, 132.37, 358.55, 159.36, 377.83, 116.95, 421.53, 167.07, 499.92, 232.61, 560.32, 300.72, 571.89]], 'area': 54652.9556, 'iscrowd': 0, 'image_id': 480023, 'bbox': [116.95, 305.86, 285.3, 266.03], 'category_id': 58, 'id': 86}
{'segmentation': [[252.46, 208.17, 267.96, 210.11, 280.14, 213.98, 285.68, 221.45, 279.59, 227.27, 270.73, 223.12, 256.61, 218.69, 253.02, 217.3, 248.03, 220.07, 245.54, 215.92, 245.82, 209.55, 249.69, 208.45]], 'area': 421.47274999999996, 'iscrowd': 0, 'image_id': 50518, 'bbox': [245.54, 208.17, 40.14, 19.1], 'category_id': 58, 'id': 89}
{'segmentation': [[292.92, 25.6, 288.4, 41.42, 289.91, 61.75, 289.16, 180.72, 314.76, 289.16, 347.14, 324.55, 356.17, 334.34, 384.79, 328.31, 391.57, 327.56, 394.58, 326.05, 402.86, 328.31, 432.98, 337.35, 429.22, 343.37, 438.

### 1. 抽取 coco 中的某一个类

In [4]:
from pycocotools.coco import COCO
import os
import shutil
from tqdm import tqdm
import skimage.io as io
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw
  
#the path you want to save your results for coco to voc
savepath="/store2/dataset/coco/yolo/" #保存提取类的路径,我放在同一路径下
img_dir=os.path.join(savepath, 'images')
anno_dir=os.path.join(savepath, 'Annotations')
# datasets_list=['train2014', 'val2014']
datasets_list=['train2014']
  
classes_names = ['book'] #coco有80类，这里写要提取类的名字，以person为例
#Store annotations and train2014/val2014/... in this folder
dataDir= '/store2/dataset/coco' #原coco数据集
  
headstr = """\
<annotation>
 <folder>VOC</folder>
 <filename>%s</filename>
 <source>
 <database>My Database</database>
 <annotation>COCO</annotation>
 <image>flickr</image>
 <flickrid>NULL</flickrid>
 </source>
 <owner>
 <flickrid>NULL</flickrid>
 <name>company</name>
 </owner>
 <size>
 <width>%d</width>
 <height>%d</height>
 <depth>%d</depth>
 </size>
 <segmented>0</segmented>
"""
objstr = """\
 <object>
 <name>%s</name>
 <pose>Unspecified</pose>
 <truncated>0</truncated>
 <difficult>0</difficult>
 <bndbox>
  <xmin>%d</xmin>
  <ymin>%d</ymin>
  <xmax>%d</xmax>
  <ymax>%d</ymax>
 </bndbox>
 </object>
"""
  
tailstr = '''\
</annotation>
'''
  
#if the dir is not exists,make it,else delete it
def mkr(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.mkdir(path)
mkr(img_dir)
mkr(anno_dir)

def id2name(coco):
    classes=dict()
    for cls in coco.dataset['categories']:
        classes[cls['id']]=cls['name']
    return classes
  
def write_xml(anno_path,head, objs, tail):
    f = open(anno_path, "w")
    f.write(head)
    for obj in objs:
        f.write(objstr%(obj[0],obj[1],obj[2],obj[3],obj[4]))
    f.write(tail)
  
  
def save_annotations_and_imgs(coco,dataset,filename,objs):
    #eg:COCO_train2014_000000196610.jpg --> COCO_train2014_000000196610.xml
    anno_path=os.path.join(anno_dir, filename[:-3]+'xml')
    img_path=os.path.join(dataDir, dataset, filename)
    dst_imgpath=os.path.join(img_dir, filename)

    img=cv2.imread(img_path)
    #if (img.shape[2] == 1):
    # print(filename + " not a RGB image")
    # return
    shutil.copy(img_path, dst_imgpath)

    head= headstr % (filename, img.shape[1], img.shape[0], img.shape[2])
    tail = tailstr
    write_xml(anno_path,head, objs, tail)
  
  
def showimg(coco,dataset,img,classes,cls_id,show=True):
    global dataDir
    I=Image.open('%s/%s/%s'%(dataDir,dataset,img['file_name']))
    #通过id，得到注释的信息
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=cls_id, iscrowd=None)
    # print(annIds)
    anns = coco.loadAnns(annIds)
    # print(anns)
    # coco.showAnns(anns)
    objs = []
    for ann in anns:
        class_name=classes[ann['category_id']]
    if class_name in classes_names:
        print(class_name)
    if 'bbox' in ann:
        bbox=ann['bbox']
        xmin = int(bbox[0])
        ymin = int(bbox[1])
        xmax = int(bbox[2] + bbox[0])
        ymax = int(bbox[3] + bbox[1])
        obj = [class_name, xmin, ymin, xmax, ymax]
        objs.append(obj)
        draw = ImageDraw.Draw(I)
        draw.rectangle([xmin, ymin, xmax, ymax])
    if show:
        plt.figure()
        plt.axis('off')
        plt.imshow(I)
        plt.show()

    return objs
  
for dataset in datasets_list:
    #./COCO/annotations/instances_train2014.json
    annFile='{}/annotations/instances_{}.json'.format(dataDir,dataset)
    print(annFile)
    #COCO API for initializing annotated data
    coco = COCO(annFile)

    #show all classes in coco
    classes = id2name(coco)
    print(classes)
    #[1, 2, 3, 4, 6, 8]
    classes_ids = coco.getCatIds(catNms=classes_names)
    print(classes_ids)
    for cls in classes_names:
    #Get ID number of this class
        cls_id=coco.getCatIds(catNms=[cls])
        img_ids=coco.getImgIds(catIds=cls_id)
    print(cls,len(img_ids))
    
# imgIds=img_ids[0:10]
for imgId in tqdm(img_ids):
    img = coco.loadImgs(imgId)[0]
    filename = img['file_name']
    # print(filename)
    objs=showimg(coco, dataset, img, classes,classes_ids,show=False)
    #print(objs)
    save_annotations_and_imgs(coco, dataset, filename, objs)

/store2/dataset/coco/annotations/instances_train2014.json
loading annotations into memory...
Done (t=8.86s)
creating index...


  0%|          | 3/3734 [00:00<02:11, 28.32it/s]

index created!
{1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'cell pho

  0%|          | 12/3734 [00:00<01:51, 33.38it/s]

book
book
book
book
book
book
book
book


...

book
book
book
book
book
book
book
book
book
book
book
book


100%|█████████▉| 3724/3734 [01:38<00:00, 46.99it/s]

book
book
book
book
book
book
book
book
book
book


100%|██████████| 3734/3734 [01:38<00:00, 37.77it/s]

book
book
book
book
book
book
